In [1]:
import os
import re
import itertools
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks
from tabulate import tabulate

### Building the dataframe with the results

In [2]:
datapath = '../data/img_labeling_1st_round/'

In [3]:
label_files = sorted(os.listdir(datapath))
label_files = [f for f in label_files if 'task_labeling' in f]
for f in label_files:
    print(f)

1st_task_labeling_Amelie_08.09.csv
1st_task_labeling_Gerda_07.09.csv
1st_task_labeling_Marcos_24.08.csv
1st_task_labeling_Renato_31.08.csv
1st_task_labeling_Yalemisew_07.09.csv
2nd_task_labeling_Amelie_08.09.csv
2nd_task_labeling_Gerda_07.09.csv
2nd_task_labeling_Marcos_28.08.csv
2nd_task_labeling_Renato_07.09.csv
2nd_task_labeling_Yalemisew_07.09.csv
3rd_task_labeling_Amelie_08.09.csv
3rd_task_labeling_Gerda_07.09.csv
3rd_task_labeling_Marcos_28.08.csv
3rd_task_labeling_Renato_07.09.csv
3rd_task_labeling_Yalemisew_08.09.csv


In [4]:
list_dfs = []
for label_file in label_files:
    task = int(re.findall(r'[1-3]{1}', label_file)[0])
    user = re.findall(r'Amelie|Gerda|Renato|Yalemisew|Marcos', label_file)[0]
    date = re.findall(r'[0-9]{2}.[0-9]{2}', label_file)[0]
    date = f'{date}.2020'
    df = pd.read_csv(f'{datapath}{label_file}', names=['class','image name', 'w','h'])
    df['task'] = task
    df['user'] = user
    df['date'] = date
    df['date'] = pd.to_datetime(df['date'])
    df.drop(['w','h'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[['task', 'user', 'date', 'image name', 'class']]
    print(f'Task: {task}, User: {user}, Images Classified: {len(df)}')
    list_dfs.append(df)

Task: 1, User: Amelie, Images Classified: 392
Task: 1, User: Gerda, Images Classified: 393
Task: 1, User: Marcos, Images Classified: 392
Task: 1, User: Renato, Images Classified: 392
Task: 1, User: Yalemisew, Images Classified: 397
Task: 2, User: Amelie, Images Classified: 392
Task: 2, User: Gerda, Images Classified: 402
Task: 2, User: Marcos, Images Classified: 392
Task: 2, User: Renato, Images Classified: 392
Task: 2, User: Yalemisew, Images Classified: 403
Task: 3, User: Amelie, Images Classified: 392
Task: 3, User: Gerda, Images Classified: 398
Task: 3, User: Marcos, Images Classified: 392
Task: 3, User: Renato, Images Classified: 392
Task: 3, User: Yalemisew, Images Classified: 399


In [5]:
df_labeling = pd.concat(list_dfs)
df_labeling['image name'].replace({'08806-~1.JPE': '08806-cgi_bin_gw_chameleon_lng_en_host_localhost_9901_DEFAULT_search_KEYWORD_function_CARDSCR_u1_12101_t1_004573881_1.jpeg', 
                                   '204804~1.JPE': '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_1C82EC077B694E00B35B9B8044A16EAB_1.jpeg',
                                   '204804~2.JPE':'2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_7000A7800599458180CD0055DB99645F_1.jpeg',
                                  }, inplace=True)
df_labeling['id_image'] = pd.Categorical(df_labeling['image name']).codes
df_labeling = df_labeling[['task', 'user', 'date', 'image name', 'id_image', 'class']]

In [6]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5920 entries, 0 to 451
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   task        5920 non-null   int64         
 1   user        5920 non-null   object        
 2   date        5920 non-null   datetime64[ns]
 3   image name  5920 non-null   object        
 4   id_image    5920 non-null   int16         
 5   class       5920 non-null   object        
dtypes: datetime64[ns](1), int16(1), int64(1), object(3)
memory usage: 289.1+ KB


In [7]:
df_labeling.head()

,task,user,date,image name,id_image,class
0,1,Amelie,2020-08-09,203-resource_document_zuiderzeemuseum_B001601_...,87,Non-Fruits
1,1,Amelie,2020-08-09,285-gam19649_1.jpeg,232,Fruits
2,1,Amelie,2020-08-09,07101-O_389_1.jpeg,6,Fruits
3,1,Amelie,2020-08-09,07101-O_927_1.jpeg,12,Fruits
4,1,Amelie,2020-08-09,07101-O_957_1.jpeg,13,Non-Fruits


In [8]:
df_labeling['class'].unique()

array(['Non-Fruits', 'Fruits', 'Informal', 'Formal', 'Appealing',
       'Non-appealing'], dtype=object)

#### Checking the numbers of images classified after fixing names:

In [9]:
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        size = len(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image'].unique())
        print(f'Task: {task}, User: {user}, Images Classified: {size}')
    print()

Task: 1, User: Amelie, Images Classified: 392
Task: 1, User: Gerda, Images Classified: 392
Task: 1, User: Marcos, Images Classified: 392
Task: 1, User: Renato, Images Classified: 392
Task: 1, User: Yalemisew, Images Classified: 392

Task: 2, User: Amelie, Images Classified: 392
Task: 2, User: Gerda, Images Classified: 392
Task: 2, User: Marcos, Images Classified: 392
Task: 2, User: Renato, Images Classified: 392
Task: 2, User: Yalemisew, Images Classified: 392

Task: 3, User: Amelie, Images Classified: 392
Task: 3, User: Gerda, Images Classified: 392
Task: 3, User: Marcos, Images Classified: 392
Task: 3, User: Renato, Images Classified: 392
Task: 3, User: Yalemisew, Images Classified: 392



#### Checking which images were not classified:

In [10]:
images = set(df_labeling.id_image.unique())
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        ids = images.difference(set(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image']))
        if len(ids) > 0:
            print(f'Missing files for task: {task}, by user: {user}')
            for _id in ids:
                print(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
            print()

#### Checking duplicates

In [11]:
duplicates = df_labeling.loc[df_labeling.duplicated(['task', 'user', 'image name', 'id_image'], keep=False), ['task', 'user', 'image name', 'id_image', 'class']]
duplicates.sort_values(by=['user', 'image name', 'task'], inplace=True)
duplicates.to_excel(f'{datapath}/duplicates.xlsx')
duplicates.head(30)

,task,user,image name,id_image,class
9,2,Gerda,07101-O_3908_1.jpeg,7,Formal
288,2,Gerda,07101-O_3908_1.jpeg,7,Informal
1,2,Gerda,07101-O_927_1.jpeg,12,Formal
286,2,Gerda,07101-O_927_1.jpeg,12,Informal
2,3,Gerda,07101-O_927_1.jpeg,12,Non-appealing
169,3,Gerda,07101-O_927_1.jpeg,12,Appealing
56,3,Gerda,2020903-KMS1518_1.jpeg,17,Non-appealing
314,3,Gerda,2020903-KMS1518_1.jpeg,17,Appealing
59,1,Gerda,2020903-KMS3015_1.jpeg,21,Fruits
317,1,Gerda,2020903-KMS3015_1.jpeg,21,Non-Fruits


### Updating the decisions on the Duplicates  

In [12]:
decisions = pd.read_csv(f'{datapath}/duplicates_decision_GK_YA.csv')
decisions.dropna(inplace=True)
decisions.drop(labels=['decision'], inplace=True, axis=1)
decisions.head()

,task,user,image name,id_image,class
0,2,Gerda,07101-O_3908_1.jpeg,7,Formal
2,2,Gerda,07101-O_927_1.jpeg,12,Formal
4,3,Gerda,07101-O_927_1.jpeg,12,Non-appealing
7,3,Gerda,2020903-KMS1518_1.jpeg,17,Appealing
9,1,Gerda,2020903-KMS3015_1.jpeg,21,Non-Fruits


In [13]:
for idx in decisions.index:
    task = decisions.loc[idx,'task'] 
    user = decisions.loc[idx,'user']
    id_image = decisions.loc[idx,'id_image']
    decision = decisions.loc[idx,'class']
    df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task) & (df_labeling.id_image == id_image), 'class'] = decision

### Analysing the [inter-annotator agreement](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) on the results

In [14]:
l1 = df_labeling.user.unique()
iter_users = list(itertools.product(l1,l1))
df_iaa = pd.DataFrame(index=l1, columns=l1)

In [15]:
for task in df_labeling.task.unique():
    for user1,user2 in iter_users:
        classesA = df_labeling.loc[(df_labeling.user == user1) & (df_labeling.task == task),['id_image', 'class']]
        classesA.sort_values(by=['id_image'], inplace=True)

        classesB = df_labeling.loc[(df_labeling.user == user2) & (df_labeling.task == task),['id_image', 'class']]
        classesB.sort_values(by=['id_image'], inplace=True)
        
        classesAB = pd.merge(classesA, classesB, on=['id_image'])
        classesAB.drop_duplicates(subset='id_image', keep = 'first', inplace=True) 
        classesAB.drop('id_image', axis=1, inplace=True)
        classesAB.dropna(inplace=True)

        agreement = cks(classesAB['class_x'], classesAB['class_y'])
        df_iaa.loc[user1,user2] = f'{agreement:.3f}/({len(classesAB)})'  
        df_iaa.index.name = f'Task_{task}'
    print(tabulate(df_iaa, headers='keys', tablefmt='psql'))
    print()

+-----------+-------------+-------------+-------------+-------------+-------------+
| Task_1    | Amelie      | Gerda       | Marcos      | Renato      | Yalemisew   |
|-----------+-------------+-------------+-------------+-------------+-------------|
| Amelie    | 1.000/(392) | 0.928/(392) | 0.892/(392) | 0.907/(392) | 0.886/(392) |
| Gerda     | 0.928/(392) | 1.000/(392) | 0.892/(392) | 0.938/(392) | 0.897/(392) |
| Marcos    | 0.892/(392) | 0.892/(392) | 1.000/(392) | 0.923/(392) | 0.923/(392) |
| Renato    | 0.907/(392) | 0.938/(392) | 0.923/(392) | 1.000/(392) | 0.918/(392) |
| Yalemisew | 0.886/(392) | 0.897/(392) | 0.923/(392) | 0.918/(392) | 1.000/(392) |
+-----------+-------------+-------------+-------------+-------------+-------------+

+-----------+--------------+-------------+--------------+--------------+--------------+
| Task_2    | Amelie       | Gerda       | Marcos       | Renato       | Yalemisew    |
|-----------+--------------+-------------+--------------+----------

In [16]:
df_labeling.info()
print(5 * 392 * 3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5920 entries, 0 to 451
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   task        5920 non-null   int64         
 1   user        5920 non-null   object        
 2   date        5920 non-null   datetime64[ns]
 3   image name  5920 non-null   object        
 4   id_image    5920 non-null   int16         
 5   class       5920 non-null   object        
dtypes: datetime64[ns](1), int16(1), int64(1), object(3)
memory usage: 289.1+ KB
5880


In [17]:
df_labeling[df_labeling.duplicated(keep=False)].sort_values(by=['id_image', 'user'])

,task,user,date,image name,id_image,class
3,3,Yalemisew,2020-08-09,07101-O_2660_1.jpeg,4,Appealing
175,3,Yalemisew,2020-08-09,07101-O_2660_1.jpeg,4,Appealing
9,2,Gerda,2020-07-09,07101-O_3908_1.jpeg,7,Formal
288,2,Gerda,2020-07-09,07101-O_3908_1.jpeg,7,Formal
4,3,Yalemisew,2020-08-09,07101-O_3908_1.jpeg,7,Non-appealing
...,...,...,...,...,...,...
269,3,Yalemisew,2020-08-09,90402-SK_A_3944_1.jpeg,331,Non-appealing
123,2,Gerda,2020-07-09,90402-SK_C_1561_1.jpeg,360,Formal
336,2,Gerda,2020-07-09,90402-SK_C_1561_1.jpeg,360,Formal
123,2,Yalemisew,2020-07-09,90402-SK_C_1561_1.jpeg,360,Informal


In [18]:
df_labeling.drop_duplicates(keep = 'first', inplace=True)

In [19]:
df_labeling['round'] = 1

In [20]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 451
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   task        5880 non-null   int64         
 1   user        5880 non-null   object        
 2   date        5880 non-null   datetime64[ns]
 3   image name  5880 non-null   object        
 4   id_image    5880 non-null   int16         
 5   class       5880 non-null   object        
 6   round       5880 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 333.0+ KB


In [21]:
df_labeling.to_hdf('../data/df_labeling.hdf', key='round1')